In [1]:
import csv
import pandas as pd
import random
import pyarrow.feather as feather
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import nltk
import itertools
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time
import gc

In [2]:
def getSample(csvstring: str):
    random.seed = 42 
    n = 11000000 #number of records in file (excludes header)
    s = 1900000 #desired sample size
    skip = sorted(random.sample(range(1,n+1),n-s))
    # Read the CSV file, skipping the randomly selected rows
    sampled_data = pd.read_csv(csvstring, on_bad_lines='skip', skiprows=skip)
    sampled_data = sampled_data.reset_index()
    return sampled_data


In [3]:
sampled_data = getSample('data/news_cleaned_2018_02_13-1.csv')


C:\Users\olink\AppData\Local\Temp\ipykernel_21416\1275764741.py:7: DtypeWarning: Columns (0,1,11,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  sampled_data = pd.read_csv(csvstring, on_bad_lines='skip', skiprows=skip)


In [4]:

len(sampled_data)
1473479



1472818

In [5]:
sampled_data.head()

,index,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,6,11,blackagendareport.com,unreliable,https://blackagendareport.com/un-backed-police...,UN-Backed Police Massacred Haitians With Impun...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,UN-Backed Police Massacred Haitians With Impunity,"Akinyele Umoja, Glen Ford, Bar Executive Edito...",NaN,[''],"“When the police finally left the campus, arou...",NaN,NaN,NaN
1,1,12,19,blackgenocide.org,conspiracy,http://blackgenocide.org/archived_articles/neg...,"""…I have set before you life and death, blessi...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,BlackGenocide.org,NaN,NaN,"['planned parenthood', 'minorities', 'black cu...",The Negro Project is a devastating eugenic sta...,NaN,NaN,NaN
2,2,23,30,blacklistednews.com,clickbait,https://www.blacklistednews.com/Egypt%E2%80%99...,Egypt’s presidential campaign has kicked into ...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Egypt’s race for president kicks off with arre...,NaN,NaN,[''],"The Best in uncensored news, information, and ...",NaN,NaN,NaN
3,3,27,34,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-conservative/...,Headline: Bitcoin & Blockchain Searches Exceed...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Surprise: Socialist Hotbed Of Venezuela Has Lo...,The Pirate'S Cove,NaN,[''],NaN,NaN,NaN,NaN
4,4,32,39,beforeitsnews.com,fake,http://beforeitsnews.com/environment/2018/01/s...,Scientists move Doomsday Clock ahead by 30 sec...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Scientists move Doomsday Clock ahead by 30 sec...,Desdemona Despair,NaN,[''],NaN,NaN,NaN,NaN


In [6]:
def removeLabels(df): 
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    df = df[(df.type == 'reliable') |(df.type == 'fake')]
    df = df[df.type.notnull()]
    df = df[df.content.notnull()]
    df = df.drop_duplicates(subset = 'content', keep = 'last')
    df = df.reset_index()
    return df 

def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict
cleanSample = removeLabels(sampled_data)
print(labelperc(cleanSample)) 
print(len(cleanSample))




{'fake': 51.69114677063201, 'reliable': 48.30885322936799}
1081899


In [7]:
del sampled_data
gc.collect()

0

In [8]:
cleanSample.content[5]

'Apparently, President Trump wondered why as well, and yesterday he decided to cut ALL military aid ($255 million per year) to Pakistan, the country that hid Osama bin Laden for years after 9/11. The monetary aid was denied to the Pakistan government in response to its ineffective action against terrorists and militants on its soil.\n\nThe Hindustan Times The Trump administration on Friday announced the United States will deny Pakistan military aid amounting to $255 million as it expects Islamabad to take decisive action “against terrorists and militants on its soil”.\n\n“The President has made clear that the United States expects Pakistan to take decisive action against terrorists and militants on its soil, and that Pakistan’s actions in support of the South Asia Strategy will ultimately determine the trajectory of our relationship, including future security assistance.\n\n“The Administration continues to review Pakistan’s level of cooperation.”\n\nThe statement reflected and sealed t

In [9]:
def checkForDuplicateArticles(df): 
    articledict = {}
    df_sample = df.sample(frac=0.1, random_state=1)
    for i in df_sample.content: 
        if str(i) in articledict: 
            articledict[str(i)] +=1
        else: 
            articledict[str(i)] = 1
    #at this point we have all different contents, and the amount of times we see them
    articledict = {key:val for key, val in articledict.items() if val >1}
    #here we delete the contents which we only see once

    contentsToRemove = list(articledict.keys())
    #get a list of the content, we will remove articles for containing
    
    
    return contentsToRemove
    
wordstoremove = checkForDuplicateArticles(cleanSample)
len(wordstoremove)




0